# 10 notebook that applies all project cards (replacing notebooks 02-09)

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [33]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

# Remote I/O and parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_08')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-04-08 17:14:51, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 17:14:51, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 17:14:51, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 17:14:51, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


# Read pickle

In [6]:
version_01_pickle_file_name = os.path.join(input_dir, 'working_scenario_01.pickle')
v_01_scenario = pickle.load(open(version_01_pickle_file_name, 'rb'))

# Create scenario and apply projects

waiting on dependencies to work

In [7]:
step1_scenario = Scenario.create_scenario(
    base_scenario=v_01_scenario,
    card_directory = card_dir,
    tags = ['highway review', "Major Arterial Review", "Reversible Lanes", 'Bus Only'],
    validate_project_cards=False)

2021-04-08 17:15:34, INFO: Creating Scenario
2021-04-08 17:15:34, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [8]:
step1_scenario.apply_all_projects()

2021-04-08 17:15:38, INFO: Applying Year 2015 Correct As Is
2021-04-08 17:15:38, INFO: Applying Year 2015 Correct As Is
2021-04-08 17:15:38, INFO: Applying Year 2015 Correct As Is
2021-04-08 17:15:38, INFO: Applying Year 2015 Correct As Is
2021-04-08 17:15:38, INFO: Applying Project to Roadway Network: Year 2015 Correct As Is
2021-04-08 17:15:38, INFO: Applying Project to Roadway Network: Year 2015 Correct As Is
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Service
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Service
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Service
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Service
2021-04-08 17:15:39, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Service
2021-04-08 17:15:39, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Service


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Ramp shape incorrect
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Ramp shape incorrect
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Ramp shape incorrect
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Ramp shape incorrect
2021-04-08 17:15:39, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Ramp shape incorrect
2021-04-08 17:15:39, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Ramp shape incorrect
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Ramp
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Ramp
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Ramp
2021-04-08 17:15:39, INFO: Applying Year 2015 Modify Facility Type to Ramp
2021-04-08 17:15:39, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to 

2021-04-08 17:15:44, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 5 Lanes
2021-04-08 17:15:44, INFO: Applying Year 2015 Should be 4 Lane should be Expressway
2021-04-08 17:15:44, INFO: Applying Year 2015 Should be 4 Lane should be Expressway
2021-04-08 17:15:44, INFO: Applying Year 2015 Should be 4 Lane should be Expressway
2021-04-08 17:15:44, INFO: Applying Year 2015 Should be 4 Lane should be Expressway
2021-04-08 17:15:44, INFO: Applying Project to Roadway Network: Year 2015 Should be 4 Lane should be Expressway
2021-04-08 17:15:44, INFO: Applying Project to Roadway Network: Year 2015 Should be 4 Lane should be Expressway
2021-04-08 17:15:44, INFO: Applying Year 2015 Should be 4 Lane should be Divided Arterial
2021-04-08 17:15:44, INFO: Applying Year 2015 Should be 4 Lane should be Divided Arterial
2021-04-08 17:15:44, INFO: Applying Year 2015 Should be 4 Lane should be Divided Arterial
2021-04-08 17:15:44, INFO: Applying Year 2015 Should be 4 Lane should be

2021-04-08 17:15:47, INFO: Applying Year 2015 Should be 3 Lane should be Undivided Arterial
2021-04-08 17:15:47, INFO: Applying Year 2015 Should be 3 Lane should be Undivided Arterial
2021-04-08 17:15:47, INFO: Applying Year 2015 Should be 3 Lane should be Undivided Arterial
2021-04-08 17:15:47, INFO: Applying Year 2015 Should be 3 Lane should be Undivided Arterial
2021-04-08 17:15:47, INFO: Applying Project to Roadway Network: Year 2015 Should be 3 Lane should be Undivided Arterial
2021-04-08 17:15:47, INFO: Applying Project to Roadway Network: Year 2015 Should be 3 Lane should be Undivided Arterial
2021-04-08 17:15:48, INFO: Applying Year 2015 Should be 2 Lane should be Undivided Arterial
2021-04-08 17:15:48, INFO: Applying Year 2015 Should be 2 Lane should be Undivided Arterial
2021-04-08 17:15:48, INFO: Applying Year 2015 Should be 2 Lane should be Undivided Arterial
2021-04-08 17:15:48, INFO: Applying Year 2015 Should be 2 Lane should be Undivided Arterial
2021-04-08 17:15:48, INF

In [9]:
step2_scenario = Scenario.create_scenario(
    base_scenario = step1_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ['Managed Lanes', 'toll review', 'Exclude Trucks'],
    validate_project_cards=False)

2021-04-08 17:15:54, INFO: Creating Scenario
2021-04-08 17:15:54, INFO: Creating Scenario


In [10]:
step2_scenario.apply_all_projects()

2021-04-08 17:15:58, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-04-08 17:15:58, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-04-08 17:15:58, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-04-08 17:15:58, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-04-08 17:15:58, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-04-08 17:15:58, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-04-08 17:15:58, INFO: Applying Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 

2021-04-08 17:16:38, INFO: Applying Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-04-08 17:16:38, INFO: Applying Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-04-08 17:16:38, INFO: Applying Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-04-08 17:16:38, INFO: Applying Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-04-08 17:16:38, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-04-08 17:16:38, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-04-08 17:16:50, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-880 to Hwy 17
2021-04-08 17:16:50, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-880 to Hwy 17
2021-04-08 17:16:50, INFO: Applying Year 2015 Modify HOV Lanes and USE on San To

2021-04-08 17:18:09, INFO: Applying Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-04-08 17:18:09, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-04-08 17:18:09, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-04-08 17:18:10, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-04-08 17:18:10, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-04-08 17:18:10, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-04-08 17:18:10, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-04-08 17:18:10, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-04-08 17:18:10, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-0

2021-04-08 17:18:41, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes USE on I-680 NB CA-242 to Benicia Bridge
2021-04-08 17:18:41, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes USE on I-680 NB CA-242 to Benicia Bridge
2021-04-08 17:18:43, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd to Rudgear Road
2021-04-08 17:18:43, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd to Rudgear Road
2021-04-08 17:18:43, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd to Rudgear Road
2021-04-08 17:18:43, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd to Rudgear Road
2021-04-08 17:18:43, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd to Rudgear Road
2021-04-08 17:18:43, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd 

2021-04-08 17:18:53, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-580 EB (segment 01) Hacienda Drive to Airway_Blvd
2021-04-08 17:18:54, INFO: Applying Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-04-08 17:18:54, INFO: Applying Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-04-08 17:18:54, INFO: Applying Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-04-08 17:18:54, INFO: Applying Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-04-08 17:18:54, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-04-08 17:18:54, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-04-08 17:18:55, INFO: Applying Year 2015 Modify HOV lanes and USE on Hwy 4 WB from De

2021-04-08 17:19:47, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-04-08 17:19:47, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-04-08 17:19:47, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-04-08 17:19:47, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-04-08 17:19:47, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-04-08 17:19:47, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-04-08 17:19:51, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 NB Bascom Ave to Magdalena Ave
2021-04-08 17:19:51, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 NB Bascom Ave to Ma

2021-04-08 17:19:59, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on San Francisco-Oakland Bay Bridge WB Toll Plaza - Bridge Pass
2021-04-08 17:19:59, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on San Francisco-Oakland Bay Bridge WB Toll Plaza - Bridge Pass
2021-04-08 17:19:59, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on San Francisco-Oakland Bay Bridge WB Toll Plaza - Bridge Pass
2021-04-08 17:19:59, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on San Francisco-Oakland Bay Bridge WB Toll Plaza - Bridge Pass
2021-04-08 17:19:59, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on San Francisco-Oakland Bay Bridge WB Toll Plaza - Bridge Pass
2021-04-08 17:20:00, INFO: Applying Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass
2021-04-08 17:20:00, INFO: Applying Year 2015 Modify toll links on Golden Ga

In [11]:
step2_scenario.road_net.links_df.shape

(1634769, 43)

In [12]:
step2_scenario.road_net.links_df.model_link_id.nunique()

1634769

In [13]:
step2_scenario.road_net.links_df.county.isnull().sum()

0

In [14]:
step2_scenario.road_net.links_df.model_link_id.max()

8063197

In [15]:
step2_scenario.road_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = step2_scenario.road_net,
    parameters = parameters)

2021-04-08 17:20:01, INFO: Adding centroid and centroid connector to standard network
2021-04-08 17:20:01, INFO: Adding centroid and centroid connector to standard network
2021-04-08 17:20:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 17:20:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 17:20:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 17:20:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 17:20:17, INFO: Finished adding centroid and centroid connectors
2021-04-08 17:20:17, INFO: Finished adding centroid and centroid connectors


In [16]:
print(step2_scenario.road_net.links_df.shape)
print(step2_scenario.road_net.links_df.model_link_id.nunique())
print(step2_scenario.road_net.nodes_df.shape)
print(step2_scenario.road_net.nodes_df.model_node_id.nunique())

(2198399, 43)
2198399
(688957, 12)
688957


In [17]:
step2_scenario.road_net.links_df["id"] = step2_scenario.road_net.links_df["shstGeometryId"]

step3_scenario = Scenario.create_scenario(
    base_scenario= step2_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ['External Stations Review'],
    validate_project_cards=False)

step3_scenario.apply_all_projects()

2021-04-08 17:20:17, INFO: Creating Scenario
2021-04-08 17:20:17, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


2021-04-08 17:20:24, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-04-08 17:20:24, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-04-08 17:20:24, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-04-08 17:20:24, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-04-08 17:20:24, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-04-08 17:20:24, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2021-04-08 17:22:45, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-04-08 17:22:45, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-04-08 17:22:45, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-04-08 17:22:45, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-04-08 17:22:45, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-04-08 17:22:45, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-04-08 17:25:07, INFO: Applying Year 2015 Add centroid connector at External Station connecting Vic Fazio Highway CA-1

2021-04-08 17:46:25, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-04-08 17:46:25, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-04-08 17:46:25, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-04-08 17:46:25, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-04-08 17:46:25, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-04-08 17:46:25, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-04-08 17:48:46, INFO: Applying Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-156 SB
2021-04-08 17:48:46, INFO: Applying Year 2015 Add centroi

2021-04-08 18:10:11, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-04-08 18:10:11, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-04-08 18:10:11, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-04-08 18:10:11, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-04-08 18:12:36, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-04-08 18:12:36, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-04-08 18:12:36, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-04-08 18:12:36, INFO

In [18]:
step4_scenario = Scenario.create_scenario(
    base_scenario= step3_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ["Major Transit links"],
    validate_project_cards=False)

step4_scenario.apply_all_projects()

2021-04-08 18:22:15, INFO: Creating Scenario
2021-04-08 18:22:15, INFO: Creating Scenario
2021-04-08 18:22:20, INFO: Applying move street rail to street connector links
2021-04-08 18:22:20, INFO: Applying move street rail to street connector links
2021-04-08 18:22:20, INFO: Applying move street rail to street connector links
2021-04-08 18:22:20, INFO: Applying move street rail to street connector links
2021-04-08 18:22:20, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-04-08 18:22:20, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-04-08 18:22:20, INFO: Applying move street rail to street connector links
2021-04-08 18:22:20, INFO: Applying move street rail to street connector links
2021-04-08 18:22:20, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-04-08 18:22:20, INFO: Applying Project to Roadway Network: move street rail to street connector links


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2021-04-08 18:24:38, INFO: Applying move street rail to street connector links
2021-04-08 18:24:38, INFO: Applying move street rail to street connector links
2021-04-08 18:24:38, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-04-08 18:24:38, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-04-08 18:26:58, INFO: Applying move street rail to street connector links
2021-04-08 18:26:58, INFO: Applying move street rail to street connector links
2021-04-08 18:26:58, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-04-08 18:26:58, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-04-08 18:29:22, INFO: Applying move street rail to street connector links
2021-04-08 18:29:22, INFO: Applying move street rail to street connector links
2021-04-08 18:29:22, INFO: Applying Project to Roadway Network: move street rail to street connect

In [19]:
step4_scenario.transit_net.set_roadnet(step4_scenario.road_net, 
                                      validate_consistency = False)

step5_scenario = Scenario.create_scenario(
    base_scenario= step4_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ["Minor Transit", "Add Transit", "Major Transit"],
    validate_project_cards=False)

step5_scenario.apply_all_projects()

2021-04-08 19:01:40, INFO: Creating Scenario
2021-04-08 19:01:40, INFO: Creating Scenario
2021-04-08 19:05:51, ERROR: Projects SFMTA Route N-JUDAH has move street rail to street connector links as prerequisite project which is missing for the scenario
2021-04-08 19:05:51, ERROR: Projects SFMTA Route N-JUDAH has move street rail to street connector links as prerequisite project which is missing for the scenario
2021-04-08 19:05:51, INFO: Applying VTA Route 22
2021-04-08 19:05:51, INFO: Applying VTA Route 22
2021-04-08 19:05:51, INFO: Applying VTA Route 22
2021-04-08 19:05:51, INFO: Applying VTA Route 22
2021-04-08 19:05:51, INFO: Applying Project to Transit Network: VTA Route 22
2021-04-08 19:05:51, INFO: Applying Project to Transit Network: VTA Route 22
2021-04-08 19:05:52, WARNING: Missing connections from node 2129493 to node 2118093 for the new routing.
2021-04-08 19:05:52, WARNING: Missing connections from node 2129493 to node 2118093 for the new routing.
2021-04-08 19:05:52, INFO:

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


2021-04-08 19:05:52, INFO: Creating a new stop in stops.txt for node ID: 2078170
2021-04-08 19:05:52, INFO: Creating a new stop in stops.txt for node ID: 2104183
2021-04-08 19:05:52, INFO: Creating a new stop in stops.txt for node ID: 2104183
2021-04-08 19:05:52, INFO: Creating a new stop in stops.txt for node ID: 2015328
2021-04-08 19:05:52, INFO: Creating a new stop in stops.txt for node ID: 2015328
2021-04-08 19:05:53, INFO: Applying VTA Route 22
2021-04-08 19:05:53, INFO: Applying VTA Route 22
2021-04-08 19:05:53, INFO: Applying Project to Transit Network: VTA Route 22
2021-04-08 19:05:53, INFO: Applying Project to Transit Network: VTA Route 22
2021-04-08 19:05:54, WARNING: Missing connections from node 2033565 to node 2103889 for the new routing.
2021-04-08 19:05:54, WARNING: Missing connections from node 2033565 to node 2103889 for the new routing.
2021-04-08 19:05:54, INFO: Routing using default graph from node 2033565 to node 2103889 for missing connections.
2021-04-08 19:05:54

2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2090983
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2090983
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2054217
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2054217
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2038694
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2038694
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2003121
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2003121
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2146572
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2146572
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2129877
2021-04-08 19:05:57, INFO: Creating a new stop in stops.txt for node ID: 2129877
2021-04-08 19:05:57, INFO: A

2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2005001
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2005001
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2111886
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2111886
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2163919
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2163919
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2059787
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2059787
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2067289
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2067289
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2018674
2021-04-08 19:06:03, INFO: Creating a new stop in stops.txt for node ID: 2018674
2021-04-08 19:06:04, INFO: A

...

2021-04-08 19:08:06, INFO: Applying SFMTA Route 14R
2021-04-08 19:08:06, INFO: Applying SFMTA Route 14R
2021-04-08 19:08:06, INFO: Applying Project to Transit Network: SFMTA Route 14R
2021-04-08 19:08:06, INFO: Applying Project to Transit Network: SFMTA Route 14R
2021-04-08 19:08:08, INFO: Creating a new stop in stops.txt for node ID: 1509377
2021-04-08 19:08:08, INFO: Creating a new stop in stops.txt for node ID: 1509377
2021-04-08 19:08:09, INFO: Applying SFMTA Route 14R
2021-04-08 19:08:09, INFO: Applying SFMTA Route 14R
2021-04-08 19:08:09, INFO: Applying SFMTA Route 14R
2021-04-08 19:08:09, INFO: Applying SFMTA Route 14R
2021-04-08 19:08:09, INFO: Applying SFMTA Route 381 Edits
2021-04-08 19:08:09, INFO: Applying SFMTA Route 381 Edits
2021-04-08 19:08:09, INFO: Applying SFMTA Route 381 Edits
2021-04-08 19:08:09, INFO: Applying SFMTA Route 381 Edits
2021-04-08 19:08:09, INFO: Applying Project to Transit Network: SFMTA Route 381 Edits
2021-04-08 19:08:09, INFO: Applying Project to T

2021-04-08 19:08:38, INFO: Applying Project to Transit Network: SFMTA Route 336 Edits
2021-04-08 19:08:40, INFO: Applying SFMTA Route 335 Edits
2021-04-08 19:08:40, INFO: Applying SFMTA Route 335 Edits
2021-04-08 19:08:40, INFO: Applying SFMTA Route 335 Edits
2021-04-08 19:08:40, INFO: Applying SFMTA Route 335 Edits
2021-04-08 19:08:40, INFO: Applying Project to Transit Network: SFMTA Route 335 Edits
2021-04-08 19:08:40, INFO: Applying Project to Transit Network: SFMTA Route 335 Edits
2021-04-08 19:08:41, INFO: Applying SFMTA Route 335 Edits
2021-04-08 19:08:41, INFO: Applying SFMTA Route 335 Edits
2021-04-08 19:08:41, INFO: Applying Project to Transit Network: SFMTA Route 335 Edits
2021-04-08 19:08:41, INFO: Applying Project to Transit Network: SFMTA Route 335 Edits
2021-04-08 19:08:42, INFO: Applying SFMTA Route 329 Edits
2021-04-08 19:08:42, INFO: Applying SFMTA Route 329 Edits
2021-04-08 19:08:42, INFO: Applying SFMTA Route 329 Edits
2021-04-08 19:08:42, INFO: Applying SFMTA Route 

2021-04-08 19:09:14, INFO: Applying SFMTA Route 304 Edits
2021-04-08 19:09:14, INFO: Applying SFMTA Route 304 Edits
2021-04-08 19:09:14, INFO: Applying Project to Transit Network: SFMTA Route 304 Edits
2021-04-08 19:09:14, INFO: Applying Project to Transit Network: SFMTA Route 304 Edits
2021-04-08 19:09:16, INFO: Applying sf bay ferry vallejo
2021-04-08 19:09:16, INFO: Applying sf bay ferry vallejo
2021-04-08 19:09:16, INFO: Applying Project to Transit Network: sf bay ferry vallejo
2021-04-08 19:09:16, INFO: Applying Project to Transit Network: sf bay ferry vallejo
2021-04-08 19:09:17, INFO: Applying sf bay ferry vallejo missing link
2021-04-08 19:09:17, INFO: Applying sf bay ferry vallejo missing link
2021-04-08 19:09:17, INFO: Applying sf bay ferry vallejo missing link
2021-04-08 19:09:17, INFO: Applying sf bay ferry vallejo missing link
2021-04-08 19:09:17, INFO: Applying Project to Roadway Network: sf bay ferry vallejo missing link
2021-04-08 19:09:17, INFO: Applying Project to Roa

2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1508126
2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1537888
2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1537888
2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1519350
2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1519350
2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1516319
2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1516319
2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1509775
2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1509775
2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1536669
2021-04-08 19:28:24, INFO: Creating a new stop in stops.txt for node ID: 1536669
2021-04-08 19:28:25, INFO: Applying SamTrans Route 393 Edits
2021-04-08 19:28:25, INFO: Applying SamTrans Rou

2021-04-08 19:28:41, INFO: Creating a new stop in stops.txt for node ID: 3024575
2021-04-08 19:28:41, INFO: Creating a new stop in stops.txt for node ID: 3069892
2021-04-08 19:28:41, INFO: Creating a new stop in stops.txt for node ID: 3069892
2021-04-08 19:28:41, INFO: Creating a new stop in stops.txt for node ID: 3018955
2021-04-08 19:28:41, INFO: Creating a new stop in stops.txt for node ID: 3018955
2021-04-08 19:28:41, INFO: Creating a new stop in stops.txt for node ID: 3029560
2021-04-08 19:28:41, INFO: Creating a new stop in stops.txt for node ID: 3029560
2021-04-08 19:28:42, INFO: Applying AC Transit Route 99 Edits
2021-04-08 19:28:42, INFO: Applying AC Transit Route 99 Edits
2021-04-08 19:28:42, INFO: Applying Project to Transit Network: AC Transit Route 99 Edits
2021-04-08 19:28:42, INFO: Applying Project to Transit Network: AC Transit Route 99 Edits
2021-04-08 19:28:43, INFO: Creating a new stop in stops.txt for node ID: 3032871
2021-04-08 19:28:43, INFO: Creating a new stop i

# sanity check

In [20]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3150085] # should be ramp lanes 2

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,ML_lanes,segment_id,tollbooth,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg,id
765904,,False,True,d1b76ff92ba7a2915de8efc97c782d73,2,,,True,,motorway_link,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350ab683bc132494cb271799a4b42450


In [24]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3107408] # bay bridge toll link

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,ML_lanes,segment_id,tollbooth,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg,id
548525,"['nan', 'nan']",False,True,826a99158fd58e5483d1085c9e0d65b3,"{'default': 2, 'timeofday': [{'time': (21600, ...","['nan', 'nan']","['nan', 'nan']","[True, True]","['I 80', 'I 80']",motorway,...,"{'default': 0, 'timeofday': [{'time': (21600, ...",5,5,5,"{'default': 0, 'timeofday': [{'time': (21600, ...",all,all,NaN,NaN,5a5d03d446a05e9bdfbdabf484f3272b


In [25]:
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 26067343
].iloc[0][["A", "B"]]

A     900020
B    2192724
Name: 2198405, dtype: object

In [26]:
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8016300].iloc[0].lanes

{'default': 3,
 'timeofday': [{'time': (21600, 36000), 'value': 4},
  {'time': (54000, 68400), 'value': 2}]}

In [27]:
step5_scenario.transit_net.feed.shapes.shape_id.nunique()

1561

In [28]:
step5_scenario.transit_net.feed.shapes[
    step5_scenario.transit_net.feed.shapes.shape_id.isin(
        ["1169", "1168", "939", "940", "943", "944"]
    )
].groupby(["shape_id"]).count()

,shape_pt_sequence,shape_osm_node_id,shape_shst_node_id,shape_model_node_id,shape_pt_lat,shape_pt_lon
shape_id,,,,,,
1168,177,108,108,177,0,0
1169,176,65,65,176,0,0
939,26,22,22,26,0,0
940,161,157,157,161,0,0
943,158,154,154,158,0,0
944,130,128,128,130,0,0


## Make Travel Model Networks

In [29]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = step5_scenario.road_net, 
    parameters = parameters)

2021-04-08 20:35:13, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:35:13, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:35:13, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:35:13, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:35:14, INFO: Filling nan for network from network wrangler
2021-04-08 20:35:14, INFO: Filling nan for network from network wrangler
2021-04-08 20:35:50, INFO: Converting variable type to mtc standard
2021-04-08 20:35:50, INFO: Converting variable type to mtc standard


In [30]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2198447, 44)
2198447
(688957, 12)
(1151833, 7)


In [35]:
model_net = mtc.add_tap_and_tap_connector(
    roadway_network = model_net,
    parameters = parameters)

2021-04-08 20:41:09, INFO: Adding tap and tap connector to standard network
2021-04-08 20:41:09, INFO: Adding tap and tap connector to standard network
2021-04-08 20:41:09, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:41:09, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:41:09, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:41:09, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
                      stop_name stop_desc   stop_lat    stop_lon zone_id  \
0  12th St. Oakland City Center       NaN  37.803664 -122.271604    12TH   
1              16th St. Mission       NaN  37.765062 -122.419694    16TH   
2              19th St. Oakland       NaN  37.807870 -122.269029    19TH   

                             stop_url  location_type parent_station  \
0  http://www.bart.gov/stations/12TH/            0.0            NaN   
1  http://www.bart.gov/statio

In [36]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.nodes_df.model_node_id.nunique())
print(model_net.shapes_df.shape)

(2233689, 44)
2233689
(694957, 12)
694957
(1169454, 7)


In [37]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2021-04-08 20:43:46, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-04-08 20:43:46, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-04-08 20:43:46, INFO: Creating managed lane network.
2021-04-08 20:43:46, INFO: Creating managed lane network.
2021-04-08 20:43:46, INFO: Creating network with duplicated managed lanes
2021-04-08 20:43:46, INFO: Creating network with duplicated managed lanes


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\wang

2021-04-08 20:59:00, INFO: Determining cntype
2021-04-08 20:59:00, INFO: Determining cntype
2021-04-08 20:59:00, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:59:00, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:59:00, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:59:00, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 20:59:00, INFO: Calculating and adding roadway network variable: cntype
2021-04-08 20:59:00, INFO: Calculating and adding roadway network variable: cntype
2021-04-08 21:00:33, INFO: Finished determining variable: cntype
2021-04-08 21:00:33, INFO: Finished determining variable: cntype
2021-04-08 21:00:33, INFO: Determining transit
2021-04-08 21:00:33, INFO: Determining transit
2021-04-08 21:00:33, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-08 21:00:33, INFO: Lasso base directory set a

In [38]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.nodes_df.model_node_id.nunique())
print(model_net.shapes_df.shape)

(2238085, 57)
2238085
(696462, 13)
696462
(1173861, 8)


In [39]:
model_net.links_df.rail_only.value_counts()

0    2237324
1        761
Name: rail_only, dtype: int64

In [40]:
model_net.links_df[(model_net.links_df.bus_only == 1)].cntype.unique()

array(['TANA'], dtype=object)

In [41]:
model_net.links_df.assignable.value_counts()

0    1845376
1     392709
Name: assignable, dtype: int64

# Write to Disk

In [42]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2021-04-09 09:01:32, INFO: Writing Network as Shapefile
2021-04-09 09:01:32, INFO: Writing Network as Shapefile
2021-04-09 09:01:32, INFO: Renaming DBF Node Variables
2021-04-09 09:01:32, INFO: Renaming DBF Node Variables
2021-04-09 09:01:32, INFO: Renaming variables so that they are DBF-safe
2021-04-09 09:01:32, INFO: Renaming variables so that they are DBF-safe
2021-04-09 09:01:44, INFO: Renaming DBF Link Variables
2021-04-09 09:01:44, INFO: Renaming DBF Link Variables
2021-04-09 09:01:44, INFO: Renaming variables so that they are DBF-safe
2021-04-09 09:01:44, INFO: Renaming variables so that they are DBF-safe
2021-04-09 09:01:47, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_08\nodes.shp
2021-04-09 09:01:47, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_08\nodes.shp
2021-04-09 09:02:32, INFO: Writing Link Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\pro

In [49]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2021-04-09 10:51:57, INFO: Starting fixed width conversion
2021-04-09 10:51:57, INFO: Starting fixed width conversion
2021-04-09 11:15:44, INFO: Writing out link database
2021-04-09 11:15:44, INFO: Writing out link database
2021-04-09 11:15:58, INFO: Writing out link header and width ----
2021-04-09 11:15:58, INFO: Writing out link header and width ----
2021-04-09 11:15:58, INFO: Starting fixed width conversion
2021-04-09 11:15:58, INFO: Starting fixed width conversion
2021-04-09 11:17:54, INFO: Writing out node database
2021-04-09 11:17:54, INFO: Writing out node database
2021-04-09 11:17:56, INFO: Writing out node header and width
2021-04-09 11:17:56, INFO: Writing out node header and width


In [50]:
%%capture
os.chdir(output_dir)
!make_cube.bat

## As transit line file (for CUBE)

In [44]:
mtc.write_cube_fare_files(
    roadway_network=step5_scenario.road_net,
    transit_network=step5_scenario.transit_net,
    parameters=parameters,
    outpath = os.path.join(output_dir)
)

In [45]:
standard_transit_net = StandardTransit.fromTransitNetwork(step5_scenario.transit_net, parameters = parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2021-04-09 09:16:48, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-09 09:16:48, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-09 09:16:48, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-09 09:16:48, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-09 09:16:48, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:16:48, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


In [48]:
for agency in standard_transit_net.feed.routes.agency_raw_name.unique():
    sub_transit_net = copy.deepcopy(standard_transit_net)
    sub_transit_net.feed.trips = sub_transit_net.feed.trips[sub_transit_net.feed.trips.agency_raw_name == agency]
    mtc.write_as_cube_lin(sub_transit_net, parameters, outpath = os.path.join(output_dir, agency + "_transit.lin"))

2021-04-09 09:28:12, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:12, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:21, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:21, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:22, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:22, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:48, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:48, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:51, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:51, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:54, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-09 09:28:54, INFO: Converting GTFS Standard Pr

## As pickle

In [52]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_08.pickle')
pickle.dump(step5_scenario, open(working_scenario_filename, 'wb'))

## As standard network

In [ ]:
step5_scenario.road_net.write(path = output_dir, filename = "v08")
step5_scenario.transit_net.write(path = output_dir, filename = "v08")